In [0]:
catalog = 'genesis_workbench'
schema = 'dev_mmt_core_test'

In [0]:
spark.sql(f"USE CATALOG {catalog}")

spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema}")

spark.sql(f"USE SCHEMA {schema}")

In [0]:
display(spark.table('genesis_workbench.dev_mmt_core_test.models'))

In [0]:
# https://adb-830292400663869.9.azuredatabricks.net/jobs/847599598308030/runs/401215867480352?o=830292400663869

spark.sql("DROP TABLE IF EXISTS models")

spark.sql(f"""
CREATE TABLE models (
    model_id BIGINT GENERATED ALWAYS AS IDENTITY,          
    model_name STRING,
    model_display_name STRING,
    model_source_version STRING,
    model_origin STRING, --uc, huggingface, pypi, bionemo, etc
    model_description_url STRING, --website to find more details about model
    model_category STRING, -- feature to which model is mapped
    model_uc_name STRING,
    model_uc_version STRING,
    model_owner STRING,
    model_added_by STRING,
    model_added_date TIMESTAMP,    
    model_input_schema STRING,
    model_output_schema STRING,
    model_params_schema STRING,
    is_model_deployed BOOLEAN,
    deployment_ids STRING,
    is_active BOOLEAN,
    deactivated_timestamp TIMESTAMP
)
""")

In [0]:
display(spark.table('genesis_workbench.dev_mmt_core_test.models'))

In [0]:
spark.sql("DROP TABLE IF EXISTS model_deployments")

spark.sql(f"""
CREATE TABLE model_deployments (
    deployment_id BIGINT,
    deployment_name STRING,
    deployment_description STRING,    
    model_id BIGINT,
    input_adapter STRING,
    output_adapter STRING,
    is_adapter BOOLEAN,
    deploy_model_uc_name STRING,
    deploy_model_uc_version STRING,
    model_deployed_date TIMESTAMP,
    model_deployed_by STRING,
    model_deploy_platform STRING, -- modelserving, dcs etc
    model_endpoint_name STRING,
    model_invoke_url STRING,
    is_active BOOLEAN,
    deactivated_timestamp TIMESTAMP
)
""")

In [0]:
# Specify the timestamp you want to query
# Jul 21, 2025, 11:04 AM
# 1295466912634444
timestamp = "2025-07-21T11:04:00.000Z"  # Replace with the desired timestamp

# Load the table at the specified timestamp
historical_df = spark.read.format("delta").option("timestampAsOf", timestamp).table("genesis_workbench.dev_mmt_core_test.models")

# Display the historical DataFrame
display(historical_df)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# # Specify the timestamp you want to query
# timestamp = "2025-07-21T11:04:00.000Z"  # Replace with the desired timestamp

# # Load the table at the specified timestamp
# historical_df = spark.read.format("delta").option("timestampAsOf", timestamp).table("genesis_workbench.dev_mmt_core_test.models")

# # Reassign model_id starting from 1
# window_spec = Window.orderBy("model_name")  # Replace "model_name" with the appropriate column to order by
# historical_df = historical_df.withColumn("model_id", row_number().over(window_spec))

# # Drop the existing models table
# spark.sql("DROP TABLE IF EXISTS genesis_workbench.dev_mmt_core_test.models")

# Save the historical DataFrame as the new models table
historical_df.write.saveAsTable('genesis_workbench.dev_mmt_core_test.models')

# Display the updated models table
display(spark.table('genesis_workbench.dev_mmt_core_test.models'))

In [0]:
display(spark.table('genesis_workbench.dev_mmt_core_test.models'))

In [0]:
# Load the table into a DataFrame
models_df = spark.table('genesis_workbench.dev_mmt_core_test.models')

# Filter out the row where model_id is 4
filtered_df = models_df.filter(models_df.model_id != 4)

# Select all columns except the identity column
columns_to_write = [col for col in filtered_df.columns if col != 'model_id']

# Delete the existing records from the table
spark.sql("DELETE FROM genesis_workbench.dev_mmt_core_test.models WHERE model_id IS NOT NULL")

# Insert the filtered DataFrame back into the table without the identity column
filtered_df.select(columns_to_write).write.mode('append').saveAsTable('genesis_workbench.dev_mmt_core_test.models')

display(spark.table('genesis_workbench.dev_mmt_core_test.models'))

In [0]:
# https://adb-830292400663869.9.azuredatabricks.net/jobs/70777228912971/runs/1035163114685297?o=830292400663869
#deployment_id 1753091718000

In [0]:
%sql

UPDATE genesis_workbench.dev_mmt_core_test.models
SET is_model_deployed = false, --true,
    deployment_ids = '' --'1753091718000'
WHERE model_name = 'SCimilarity_Search_Nearest';

In [0]:
display(spark.table('genesis_workbench.dev_mmt_core_test.models'))

In [0]:
display(spark.table('genesis_workbench.dev_mmt_core_test.model_deployments'))